In [1]:
# Load in libraries and plot parameters
import pandas as pd
import numpy as np
import os
from scipy import fftpack, signal, interpolate
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib.colors import Normalize, LogNorm
from matplotlib.cm import ScalarMappable
import sys
sys.path.append('./py-nbs')

# Importing functions from gw_utils and noise_curves
from utilsgw import (
    compute_fft, compute_asd, butterworth_filter, model_list, resample_signal, taper_signal,
    plot_waveforms, make_combined_plot, style_plot, 
    calculate_snr_waveform, calculate_partial_snr, plot_sensitivity_curves
)
from noise_curves import (
    Asharp, LIGO_O3b_1262178304, AdVirgo, LIGO_O3_H1, 
    LIGO_O3_L1, ALIGO_O4, CE, ET, LGWAnb, LGWAsi, GLOCc, GLOCo, 
    DECIGO, BDECIGO, LILA, ALIA, DO, TianGO, BBO, atom
)

# Matplotlib parameters
plt.rcParams.update({
    "figure.figsize": (41, 24),
    "font.size": 38,
    "xtick.direction": 'in',
    "ytick.direction": 'in',
    "xtick.top": True,
    "ytick.right": True,
    "pgf.texsystem": 'pdflatex',
    "pgf.rcfonts": False,
    "axes.linewidth": 4,
    "xtick.major.size": 12,
    "xtick.major.width": 4,
    "xtick.minor.size": 10,
    "xtick.minor.width": 3,
    "ytick.major.size": 12,
    "ytick.major.width": 4,
    "ytick.minor.size": 10,
    "ytick.minor.width": 3
})

In [2]:
## TOTAL MATTER SNR
models = ['9a', '9b', '9.25', '9.5', '11', '12.25', '14', '15.01', '16', '17', '18', '18.5', '19', '19.56', '20', '21.68', '23', '24', '25', '40', '60']
factor = 1 / 3.086e22  # Convert 10 kpc to centimeters

# Define the base directory
base_dir = os.path.abspath('..')
matter_data_dir = os.path.join(base_dir, 'py-nbs','Data', 'Models', 'Matter')

## Define lists for SNR results
snriLILA_list = []
snrGLOCo_list = []
snrsi_list = []
snrO3H1_list = []
snrA_list = []
snrCE_list = []
snrET_list = []
snrDECIGO_list = []
snrBDECIGO_list = []
snrALIA_list = []
snrDO_list = []
snrBBO_list = []
snrTianGO_list = []
snratom_list = []
for model_index, model in enumerate(models):
    try:
        file_path = os.path.join(matter_data_dir, f'{model}_matter_processed_data.txt')
        time, hp = np.loadtxt(file_path, usecols=(0, 1), unpack=True, skiprows=1)
        
        # Apply factor to hp
        hp_scaled = hp * factor

        # Calculate SNR for each detector
        snriLILA = calculate_snr_waveform(time, hp_scaled, LILA[:, 0], LILA[:, 1])
        snrGLOCo = calculate_snr_waveform(time, hp_scaled, GLOCo[:, 0], GLOCo[:, 1])
        snrsi = calculate_snr_waveform(time, hp_scaled, LGWAsi[:, 0], np.sqrt(LGWAsi[:, 1]))
        snrO3 = calculate_snr_waveform(time, hp_scaled, LIGO_O3_H1[:, 0], LIGO_O3_H1[:, 1])
        snrA = calculate_snr_waveform(time, hp_scaled, Asharp[:, 0], Asharp[:, 1])
        snrCE = calculate_snr_waveform(time, hp_scaled, CE[:, 0], np.sqrt(CE[:, 1]))
        snrET = calculate_snr_waveform(time, hp_scaled, ET[:, 0], ET[:, 1])
        snrDECIGO = calculate_snr_waveform(time, hp_scaled, DECIGO[:, 0], DECIGO[:, 1])
        snrBDECIGO = calculate_snr_waveform(time, hp_scaled, BDECIGO[:, 0], BDECIGO[:, 1])
        snrALIA = calculate_snr_waveform(time, hp_scaled, ALIA[:, 0], ALIA[:, 1])
        snrDO = calculate_snr_waveform(time, hp_scaled, DO[:, 0], DO[:, 1])
        snrBBO = calculate_snr_waveform(time, hp_scaled, BBO[:, 0], BBO[:, 1])
        snrTianGO = calculate_snr_waveform(time, hp_scaled, TianGO[:, 0], TianGO[:, 1])
        snratom = calculate_snr_waveform(time, hp_scaled, atom[:, 0], atom[:, 1])
        # Append results to lists
        snriLILA_list.append(snriLILA)
        snrGLOCo_list.append(snrGLOCo)
        snrsi_list.append(snrsi)
        snrO3H1_list.append(snrO3)
        snrA_list.append(snrA)
        snrCE_list.append(snrCE)
        snrET_list.append(snrET)
        snrDECIGO_list.append(snrDECIGO)
        snrBDECIGO_list.append(snrBDECIGO)
        snrALIA_list.append(snrALIA)
        snrDO_list.append(snrDO)
        snrBBO_list.append(snrBBO)
        snrTianGO_list.append(snrTianGO)
        snratom_list.append(snratom)
    except Exception as e:
        print(f"Error loading data for model {model}: {e}")
        continue

# Ensure lists are of the same length before creating the DataFrame
length = min(len(snriLILA_list), len(snrGLOCo_list), len(snrsi_list), len(snrO3H1_list), len(snrA_list), 
             len(snrCE_list), len(snrET_list), len(snrDECIGO_list), len(snrBDECIGO_list), 
             len(snrALIA_list), len(snrDO_list), len(snrBBO_list), len(snrTianGO_list),len(snratom_list) )

df_matter = pd.DataFrame()
df_matter['Model'] = models[:length]
df_matter['LILA'] = snriLILA_list[:length]
df_matter['GLOCo'] = snrGLOCo_list[:length]
df_matter['LGWAsi'] = snrsi_list[:length]
df_matter['LIGO3'] = snrO3H1_list[:length]
df_matter['Asharp'] = snrA_list[:length]
df_matter['CE'] = snrCE_list[:length]
df_matter['ET'] = snrET_list[:length]
df_matter['DECIGO'] = snrDECIGO_list[:length]
df_matter['BDECIGO'] = snrBDECIGO_list[:length]
df_matter['ALIA'] = snrALIA_list[:length]
df_matter['DO'] = snrDO_list[:length]
df_matter['BBO'] = snrBBO_list[:length]
df_matter['TianGO'] = snrTianGO_list[:length]
df_matter['Atomic Clock'] = snratom_list[:length]
df_matter


,Model,LILA,GLOCo,LGWAsi,LIGO3,Asharp,CE,ET,DECIGO,BDECIGO,ALIA,DO,BBO,TianGO,Atomic Clock
0,9a,0.146625,31.751076,0.019984,0.931422,5.920292,15.157418,16.118731,11.834040,0.520340,0.514226,2.098143,2.975426,0.074660,0.135810
1,9b,0.165743,45.848963,0.003722,1.397856,8.163149,21.609684,22.799536,1.496253,0.105493,0.039678,0.240615,0.644946,0.020996,0.010508
2,9.25,0.408106,63.175562,0.016707,1.674893,12.031469,30.769350,30.195197,6.570490,0.559312,0.204510,1.082432,3.413787,0.063255,0.054055
3,9.5,0.460162,100.503977,0.080500,2.824348,17.914439,48.018140,46.555212,49.366173,2.054083,2.183038,8.791721,11.572094,0.297080,0.576519
4,11,1.528805,181.113555,0.144948,4.662376,32.928899,87.581428,80.938487,110.792381,3.775843,5.476716,20.318703,18.620546,0.518762,1.445814
5,12.25,0.072411,109.942087,0.000784,2.716206,20.453370,52.055240,49.377921,0.552751,0.021961,0.025778,0.099044,0.617455,0.020099,0.006806
6,14,0.374096,190.729222,0.018068,4.823297,30.413816,83.481849,79.232770,9.424460,0.509685,0.386915,1.644840,3.012417,0.102125,0.102211
7,15.01,1.826482,247.855425,0.169208,5.647213,42.849294,112.192961,101.366673,123.579481,4.453185,6.059609,22.600874,22.772812,0.628197,1.599655
8,16,1.427951,352.339760,0.157137,8.925899,53.538806,163.368600,144.071386,74.158906,4.180341,2.756318,12.675307,24.884941,0.562328,0.728537
9,17,3.991112,534.076233,0.979682,12.807214,80.541536,235.314941,208.883343,570.758362,25.304506,24.643473,101.025493,144.578741,3.549017,6.508878


In [3]:
## Partial MATTER SNR
models = ['9a', '9b', '9.25', '9.5', '11', '12.25', '14', '15.01', '16', '17', '18', '18.5', '19', '19.56', '20', '21.68', '23', '24', '25', '40', '60']
factor = 1 / 3.086e22  # Convert 10 kpc to centimeters

# Define the base directory
base_dir = os.path.abspath('..')
matter_data_dir = os.path.join(base_dir, 'py-nbs','Data', 'Models', 'Matter')

## Define lists for SNR results
snriLILA_list = []
snrGLOCo_list = []
snrsi_list = []
snrO3H1_list = []
snrA_list = []
snrCE_list = []
snrET_list = []
snrDECIGO_list = []
snrBDECIGO_list = []
snrALIA_list = []
snrDO_list = []
snrBBO_list = []
snrTianGO_list = []
snratom_list = []
for model_index, model in enumerate(models):
    try:
        file_path = os.path.join(matter_data_dir, f'{model}_matter_processed_data.txt')
        time, hp = np.loadtxt(file_path, usecols=(0, 1), unpack=True, skiprows=1)
        
        # Apply factor to hp
        hp_scaled = hp * factor

        # Calculate SNR for each detector
        snriLILA = calculate_partial_snr(time, hp_scaled, LILA[:, 0], LILA[:, 1])
        snrGLOCo = calculate_partial_snr(time, hp_scaled, GLOCo[:, 0], GLOCo[:, 1])
        snrsi = calculate_partial_snr(time, hp_scaled, LGWAsi[:, 0], np.sqrt(LGWAsi[:, 1]))
        snrO3 = calculate_partial_snr(time, hp_scaled, LIGO_O3_H1[:, 0], LIGO_O3_H1[:, 1])
        snrA = calculate_partial_snr(time, hp_scaled, Asharp[:, 0], Asharp[:, 1])
        snrCE = calculate_partial_snr(time, hp_scaled, CE[:, 0], np.sqrt(CE[:, 1]))
        snrET = calculate_partial_snr(time, hp_scaled, ET[:, 0], ET[:, 1])
        snrDECIGO = calculate_partial_snr(time, hp_scaled, DECIGO[:, 0], DECIGO[:, 1])
        snrBDECIGO = calculate_partial_snr(time, hp_scaled, BDECIGO[:, 0], BDECIGO[:, 1])
        snrALIA = calculate_partial_snr(time, hp_scaled, ALIA[:, 0], ALIA[:, 1])
        snrDO = calculate_partial_snr(time, hp_scaled, DO[:, 0], DO[:, 1])
        snrBBO = calculate_snr_waveform(time, hp_scaled, BBO[:, 0], BBO[:, 1])
        snrTianGO = calculate_snr_waveform(time, hp_scaled, TianGO[:, 0], TianGO[:, 1])
        snratom = calculate_snr_waveform(time, hp_scaled, atom[:, 0], atom[:, 1])

        # Append results to lists
        snriLILA_list.append(snriLILA)
        snrGLOCo_list.append(snrGLOCo)
        snrsi_list.append(snrsi)
        snrO3H1_list.append(snrO3)
        snrA_list.append(snrA)
        snrCE_list.append(snrCE)
        snrET_list.append(snrET)
        snrDECIGO_list.append(snrDECIGO)
        snrBDECIGO_list.append(snrBDECIGO)
        snrALIA_list.append(snrALIA)
        snrDO_list.append(snrDO)
        snrBBO_list.append(snrBBO)
        snrTianGO_list.append(snrTianGO)
        snratom_list.append(snratom)
    except Exception as e:
        print(f"Error loading data for model {model}: {e}")
        continue

# Ensure lists are of the same length before creating the DataFrame
length = min(len(snriLILA_list), len(snrGLOCo_list), len(snrsi_list), len(snrO3H1_list), len(snrA_list), 
             len(snrCE_list), len(snrET_list), len(snrDECIGO_list), len(snrBDECIGO_list), 
             len(snrALIA_list), len(snrDO_list), len(snrBBO_list), len(snrTianGO_list),len(snratom_list))

df_matter = pd.DataFrame()
df_matter['Model'] = models[:length]
df_matter['LILA'] = snriLILA_list[:length]
df_matter['GLOCo'] = snrGLOCo_list[:length]
df_matter['LGWAsi'] = snrsi_list[:length]
df_matter['LIGO3'] = snrO3H1_list[:length]
df_matter['Asharp'] = snrA_list[:length]
df_matter['CE'] = snrCE_list[:length]
df_matter['ET'] = snrET_list[:length]
df_matter['DECIGO'] = snrDECIGO_list[:length]
df_matter['BDECIGO'] = snrBDECIGO_list[:length]
df_matter['ALIA'] = snrALIA_list[:length]
df_matter['DO'] = snrDO_list[:length]
df_matter['BBO'] = snrBBO_list[:length]
df_matter['TianGO'] = snrTianGO_list[:length]
df_matter['Atomic Clock'] = snratom_list[:length]
df_matter


,Model,LILA,GLOCo,LGWAsi,LIGO3,Asharp,CE,ET,DECIGO,BDECIGO,ALIA,DO,BBO,TianGO,Atomic Clock
0,9a,0.144026,1.059569,0.016077,0.000003,0.005137,0.075412,0.368788,6.432591,0.421615,0.076158,1.007970,2.975426,0.074660,0.135810
1,9b,0.145204,1.368676,0.003618,0.000003,0.005838,0.089254,0.463112,1.287094,0.103041,0.013260,0.198917,0.644946,0.020996,0.010508
2,9.25,0.403999,1.920929,0.016041,0.000004,0.007314,0.119473,0.585440,5.277090,0.546287,0.042586,0.804720,3.413787,0.063255,0.054055
3,9.5,0.427787,3.179864,0.062738,0.000006,0.014745,0.215067,1.018617,25.594293,1.593725,0.308462,4.018004,11.572094,0.297080,0.576519
4,11,1.522778,5.340208,0.068877,0.000008,0.015765,0.255822,1.465414,29.681932,1.876472,0.370152,4.680173,18.620546,0.518762,1.445814
5,12.25,0.020984,0.323554,0.000507,0.000001,0.001885,0.031559,0.107471,0.220394,0.015645,0.002647,0.034643,0.617455,0.020099,0.006806
6,14,0.354771,2.851122,0.015713,0.000005,0.010922,0.161974,0.874408,5.745093,0.455247,0.060973,0.890789,3.012417,0.102125,0.102211
7,15.01,1.811367,12.302988,0.093395,0.000017,0.039424,0.567520,3.389498,36.409228,2.586563,0.415431,5.683380,22.772812,0.628197,1.599655
8,16,1.396088,13.128626,0.143958,0.000023,0.053323,0.762373,4.118972,52.075613,3.854273,0.562594,8.079989,24.884941,0.562328,0.728537
9,17,3.939234,17.666567,0.797759,0.000032,0.063778,0.978045,4.935272,315.134557,20.661445,3.709874,49.338998,144.578741,3.549017,6.508878


In [4]:
## TOTAL NEUTRINO SNR
models = ['9a', '9b', '9.25', '9.5', '11', '12.25', '14', '15.01', '16', '17', '18', '18.5', '19', '19.56', '20', '21.68', '23', '24', '25', '40', '60']
factor = 1 / 3.086e22  # Convert 10 kpc to centimeters

# Define the base directory
base_dir = os.path.abspath('..')
neutrino_data_dir = os.path.join(base_dir, 'py-nbs','Data', 'Models', 'Neutrino')

## Define lists for SNR results
snriLILA_list = []
snrGLOCo_list = []
snrsi_list = []
snrO3H1_list = []
snrA_list = []
snrCE_list = []
snrET_list = []
snrDECIGO_list = []
snrBDECIGO_list = []
snrALIA_list = []
snrDO_list = []
snrBBO_list = []
snrTianGO_list = []
snratom_list = []
for model_index, model in enumerate(models):
    try:
        file_path = os.path.join(neutrino_data_dir, f'{model}_neutrino_processed_data.txt')
        time, hp = np.loadtxt(file_path, usecols=(0, 1), unpack=True, skiprows=1)
        
        # Apply factor to hp
        hp_scaled = hp * factor

        # Calculate SNR for each detector
        snriLILA = calculate_snr_waveform(time, hp_scaled, LILA[:, 0], LILA[:, 1])
        snrGLOCo = calculate_snr_waveform(time, hp_scaled, GLOCo[:, 0], GLOCo[:, 1])
        snrsi = calculate_snr_waveform(time, hp_scaled, LGWAsi[:, 0], np.sqrt(LGWAsi[:, 1]))
        snrO3 = calculate_snr_waveform(time, hp_scaled, LIGO_O3_H1[:, 0], LIGO_O3_H1[:, 1])
        snrA = calculate_snr_waveform(time, hp_scaled, Asharp[:, 0], Asharp[:, 1])
        snrCE = calculate_snr_waveform(time, hp_scaled, CE[:, 0], np.sqrt(CE[:, 1]))
        snrET = calculate_snr_waveform(time, hp_scaled, ET[:, 0], ET[:, 1])
        snrDECIGO = calculate_snr_waveform(time, hp_scaled, DECIGO[:, 0], DECIGO[:, 1])
        snrBDECIGO = calculate_snr_waveform(time, hp_scaled, BDECIGO[:, 0], BDECIGO[:, 1])
        snrALIA = calculate_snr_waveform(time, hp_scaled, ALIA[:, 0], ALIA[:, 1])
        snrDO = calculate_snr_waveform(time, hp_scaled, DO[:, 0], DO[:, 1])
        snrBBO = calculate_snr_waveform(time, hp_scaled, BBO[:, 0], BBO[:, 1])
        snrTianGO = calculate_snr_waveform(time, hp_scaled, TianGO[:, 0], TianGO[:, 1])
        snratom = calculate_snr_waveform(time, hp_scaled, atom[:, 0], atom[:, 1])

        # Append results to lists
        snriLILA_list.append(snriLILA)
        snrGLOCo_list.append(snrGLOCo)
        snrsi_list.append(snrsi)
        snrO3H1_list.append(snrO3)
        snrA_list.append(snrA)
        snrCE_list.append(snrCE)
        snrET_list.append(snrET)
        snrDECIGO_list.append(snrDECIGO)
        snrBDECIGO_list.append(snrBDECIGO)
        snrALIA_list.append(snrALIA)
        snrDO_list.append(snrDO)
        snrBBO_list.append(snrBBO)
        snrTianGO_list.append(snrTianGO)
        snratom_list.append(snratom)
    except Exception as e:
        print(f"Error loading data for model {model}: {e}")
        continue

# Ensure lists are of the same length before creating the DataFrame
length = min(len(snriLILA_list), len(snrGLOCo_list), len(snrsi_list), len(snrO3H1_list), len(snrA_list), 
             len(snrCE_list), len(snrET_list), len(snrDECIGO_list), len(snrBDECIGO_list), 
             len(snrALIA_list), len(snrDO_list), len(snrBBO_list), len(snrTianGO_list), len(snratom_list))

df_neutrino = pd.DataFrame()
df_neutrino['Model'] = models[:length]
df_neutrino['LILA'] = snriLILA_list[:length]
df_neutrino['GLOCo'] = snrGLOCo_list[:length]
df_neutrino['LGWAsi'] = snrsi_list[:length]
df_neutrino['LIGO3'] = snrO3H1_list[:length]
df_neutrino['Asharp'] = snrA_list[:length]
df_neutrino['CE'] = snrCE_list[:length]
df_neutrino['ET'] = snrET_list[:length]
df_neutrino['DECIGO'] = snrDECIGO_list[:length]
df_neutrino['BDECIGO'] = snrBDECIGO_list[:length]
df_neutrino['ALIA'] = snrALIA_list[:length]
df_neutrino['DO'] = snrDO_list[:length]
df_neutrino['BBO'] = snrBBO_list[:length]
df_neutrino['TianGO'] = snrTianGO_list[:length]
df_neutrino['Atomic Clock'] = snratom_list[:length]
df_neutrino


,Model,LILA,GLOCo,LGWAsi,LIGO3,Asharp,CE,ET,DECIGO,BDECIGO,ALIA,DO,BBO,TianGO,Atomic Clock
0,9a,14.439395,117.887911,0.897328,0.390275,3.223626,32.931511,34.964681,417.236972,24.903956,15.263913,71.108569,149.222029,3.034343,4.033304
1,9b,13.770512,124.215157,0.593399,0.493279,4.065553,41.200657,29.428400,249.634209,16.529529,7.936664,41.376857,99.718045,1.991410,2.099119
2,9.25,25.178277,226.686585,1.468649,1.226093,8.901339,70.119102,62.732090,651.745349,41.484587,20.899756,108.756304,253.580391,4.931737,5.529233
3,9.5,46.584415,352.756110,2.181949,1.165776,8.626825,74.696149,94.440314,963.361056,61.049794,33.126271,162.248326,366.652144,7.291483,8.756084
4,11,66.194700,412.907062,37.978096,1.749823,13.451745,124.281810,100.623789,25354.246413,961.488084,1175.239994,4570.263950,5202.832146,142.098745,310.349954
5,12.25,47.845796,433.439272,9.412033,3.462520,22.688178,150.606110,117.239390,6122.257215,238.010516,277.315083,1097.363954,1315.188775,35.102751,73.233948
6,14,119.718883,742.481901,7.267773,5.482643,36.725124,252.592092,198.455373,5314.672452,194.714788,256.756517,968.591542,1021.165979,25.791826,67.786469
7,15.01,140.178705,903.312243,11.644038,2.907895,22.856056,204.783113,224.793648,6835.865351,303.251376,275.060674,1194.113172,1785.863214,39.846482,72.702767
8,16,97.984098,517.329091,16.330041,2.547167,18.158829,139.731578,126.787759,12210.660502,436.698888,611.717792,2246.237781,2163.468719,57.432114,161.488645
9,17,183.954010,919.072866,20.634585,3.743878,27.270377,229.982382,258.922114,8887.323331,574.645854,298.205052,1490.441812,3460.579665,68.333210,78.848979


In [5]:
## Partial NEUTRINO SNR
models = ['9a', '9b', '9.25', '9.5', '11', '12.25', '14', '15.01', '16', '17', '18', '18.5', '19', '19.56', '20', '21.68', '23', '24', '25', '40', '60']
factor = 1 / 3.086e22  # Convert 10 kpc to centimeters

# Define the base directory
base_dir = os.path.abspath('..')
neutrino_data_dir = os.path.join(base_dir, 'py-nbs','Data', 'Models', 'Neutrino')

## Define lists for SNR results
snriLILA_list = []
snrGLOCo_list = []
snrsi_list = []
snrO3H1_list = []
snrA_list = []
snrCE_list = []
snrET_list = []
snrDECIGO_list = []
snrBDECIGO_list = []
snrALIA_list = []
snrDO_list = []
snrBBO_list = []
snrTianGO_list = []
snratom_list = []
for model_index, model in enumerate(models):
    try:
        file_path = os.path.join(neutrino_data_dir, f'{model}_neutrino_processed_data.txt')
        time, hp = np.loadtxt(file_path, usecols=(0, 1), unpack=True, skiprows=1)
        
        # Apply factor to hp
        hp_scaled = hp * factor

        # Calculate SNR for each detector
        snriLILA = calculate_partial_snr(time, hp_scaled, LILA[:, 0], LILA[:, 1])
        snrGLOCo = calculate_partial_snr(time, hp_scaled, GLOCo[:, 0], GLOCo[:, 1])
        snrsi = calculate_partial_snr(time, hp_scaled, LGWAsi[:, 0], np.sqrt(LGWAsi[:, 1]))
        snrO3 = calculate_partial_snr(time, hp_scaled, LIGO_O3_H1[:, 0], LIGO_O3_H1[:, 1])
        snrA = calculate_partial_snr(time, hp_scaled, Asharp[:, 0], Asharp[:, 1])
        snrCE = calculate_partial_snr(time, hp_scaled, CE[:, 0], np.sqrt(CE[:, 1]))
        snrET = calculate_partial_snr(time, hp_scaled, ET[:, 0], ET[:, 1])
        snrDECIGO = calculate_partial_snr(time, hp_scaled, DECIGO[:, 0], DECIGO[:, 1])
        snrBDECIGO = calculate_partial_snr(time, hp_scaled, BDECIGO[:, 0], BDECIGO[:, 1])
        snrALIA = calculate_partial_snr(time, hp_scaled, ALIA[:, 0], ALIA[:, 1])
        snrDO = calculate_partial_snr(time, hp_scaled, DO[:, 0], DO[:, 1])
        snrTianGO = calculate_partial_snr(time, hp_scaled, TianGO[:, 0], TianGO[:, 1])
        snrBBO = calculate_partial_snr(time, hp_scaled, BBO[:, 0], BBO[:, 1])
        snratom = calculate_partial_snr(time, hp_scaled, atom[:, 0], atom[:, 1])

        # Append results to lists
        snriLILA_list.append(snriLILA)
        snrGLOCo_list.append(snrGLOCo)
        snrsi_list.append(snrsi)
        snrO3H1_list.append(snrO3)
        snrA_list.append(snrA)
        snrCE_list.append(snrCE)
        snrET_list.append(snrET)
        snrDECIGO_list.append(snrDECIGO)
        snrBDECIGO_list.append(snrBDECIGO)
        snrALIA_list.append(snrALIA)
        snrDO_list.append(snrDO)
        snrBBO_list.append(snrBBO)
        snrTianGO_list.append(snrTianGO)
        snratom_list.append(snratom)
    except Exception as e:
        print(f"Error loading data for model {model}: {e}")
        continue

# Ensure lists are of the same length before creating the DataFrame
length = min(len(snriLILA_list), len(snrGLOCo_list), len(snrsi_list), len(snrO3H1_list), len(snrA_list), 
             len(snrCE_list), len(snrET_list), len(snrDECIGO_list), len(snrBDECIGO_list), 
             len(snrALIA_list), len(snrDO_list), len(snrBBO_list), len(snrTianGO_list),len(snratom_list))

df_neutrino = pd.DataFrame()
df_neutrino['Model'] = models[:length]
df_neutrino['LILA'] = snriLILA_list[:length]
df_neutrino['GLOCo'] = snrGLOCo_list[:length]
df_neutrino['LGWAsi'] = snrsi_list[:length]
df_neutrino['LIGO3'] = snrO3H1_list[:length]
df_neutrino['Asharp'] = snrA_list[:length]
df_neutrino['CE'] = snrCE_list[:length]
df_neutrino['ET'] = snrET_list[:length]
df_neutrino['DECIGO'] = snrDECIGO_list[:length]
df_neutrino['BDECIGO'] = snrBDECIGO_list[:length]
df_neutrino['ALIA'] = snrALIA_list[:length]
df_neutrino['DO'] = snrDO_list[:length]
df_neutrino['BBO'] = snrBBO_list[:length]
df_neutrino['TianGO'] = snrTianGO_list[:length]
df_neutrino['Atomic Clock'] = snratom_list[:length]
df_neutrino

,Model,LILA,GLOCo,LGWAsi,LIGO3,Asharp,CE,ET,DECIGO,BDECIGO,ALIA,DO,BBO,TianGO,Atomic Clock
0,9a,14.435632,91.738872,0.826896,0.000110,0.342162,4.335192,30.443512,298.345893,23.238930,3.165037,46.234322,143.933823,2.730184,0.857663
1,9b,13.630590,81.703527,0.565744,0.000069,0.163741,2.152793,20.335255,198.613286,15.876377,2.041861,30.684815,97.643194,1.871600,0.553513
2,9.25,25.132172,156.882676,1.391509,0.000319,0.509246,8.022984,48.115019,519.677060,39.690069,5.665010,80.778048,247.963171,4.597459,1.534605
3,9.5,46.485770,303.231003,2.048063,0.000425,0.796335,11.667224,83.170072,728.703521,57.906677,7.594819,112.750632,356.655723,6.707395,2.058289
4,11,66.092727,296.895263,26.454921,0.000349,0.773524,10.424793,73.483600,11085.583041,659.102171,136.122586,1744.187470,4194.362221,97.326812,36.854359
5,12.25,47.810402,254.440228,6.885836,0.000535,1.056540,17.280443,72.746608,2940.732295,171.682117,36.480090,463.313411,1097.364672,25.254358,9.875884
6,14,119.651704,477.877830,4.144374,0.000795,1.583531,22.940195,138.269162,1812.002674,119.947730,22.228946,285.411587,766.686497,12.271659,6.016433
7,15.01,140.106977,771.527802,9.797660,0.000722,2.288639,31.160347,195.177892,4359.396794,256.577027,55.105597,688.755956,1638.287494,31.877333,14.914522
8,16,97.866406,411.023470,7.941450,0.000480,1.200369,16.142787,101.681894,2552.984795,237.327438,22.883270,389.910363,1442.856807,18.944534,6.214555
9,17,183.731960,744.966508,19.495117,0.001232,2.575290,34.867253,222.405681,6852.826686,547.756680,70.872787,1059.430714,3375.196894,63.318604,19.208971
